In [20]:
import os
import re
import time
import pickle
import torch
import torchvision
import torch.nn as nn
import torch.optim as optim
import numpy as np
import pandas as pd
from collections import OrderedDict
from net.st_gcn import Model
from feeder.feeder import Feeder
from torchlight.io import IO
import matplotlib.pyplot as plt


In [21]:
# Loading of original weights
root_path = './'
weights_path =  './models/st_gcn.kinetics.pt'
io = IO(root_path)
model_name = 'net.st_gcn.Model'
graph_args = {'layout':'openpose', 'strategy':'spatial'}
model_args = {
    'edge_importance_weighting':True,
    'graph_args':graph_args,
    'in_channels':3,
    'num_class':400, }


In [22]:
model = io.load_model(model_name, **model_args).to('cuda')
# Need to turn into a func
weights = torch.load(weights_path)
weights_list = [[k.split('module.')[-1], v.cpu()] for k, v in weights.items()]
weights = OrderedDict(weights_list)
model_state_dict = model.state_dict()
weights.pop('fcn.bias') # loading all but the Final FC layer's weight and bias
weights.pop('fcn.weight')
model_state_dict.update(weights)
model.load_state_dict(model_state_dict)

<All keys matched successfully>

In [23]:
featureFolder='./data/urfall/feature/1125_2048/'
# cerate folder if not exist
if not os.path.exists(featureFolder):
    os.makedirs(featureFolder)
adlSkeleton = './data/urfall/jointNumpy/adl_Data1124.npy'


data=np.load(adlSkeleton, allow_pickle=True)
print (data.shape)
# (40, 300, 18, 3)
# (30, 300, 18, 3)
featureList=[]
# featureList30=[]
# featureList40=[]
# featureList50=[]
# featureList60=[]
# featureList70=[]
# featureList80=[]
# featureList90=[]
# allFeatureList=[featureList30, featureList40, featureList50, featureList60, featureList70, featureList80, featureList90]
# nList=[30, 40, 50, 60, 70, 80, 90]

for i in range(data.shape[0]):
    print (i, '......')

    caseData=data[i]   # 300, 18,3 

    # remove the frames with all zeros
    caseData=caseData.reshape(caseData.shape[0], -1)
    caseData=caseData[~np.all(caseData == 0, axis=1)]
    caseData=caseData.reshape(-1, 18, 3)
    print (caseData.shape)

    length=caseData.shape[0]
    for n in range(0,length, 10):
    
        subdata=caseData[0:n, :, :]
        padding = np.zeros((300-n, 18, 3))
        subdata = np.concatenate((subdata, padding), axis=0)
        print (subdata.shape)
        subdata= subdata.reshape(1, 300, 18, 3)
        subdata=subdata.transpose(0, 3, 1, 2).reshape(subdata.shape[0], 3,300, 18, 1)
        subdata= torch.from_numpy(subdata).float()
        out, feature= model.extract_feature(subdata.to('cuda'))
        featureList.append(feature.cpu().cpu().detach().numpy())

featureList=np.array(featureList)
np.save(featureFolder + 'adl_feature_expand_1125.npy', featureList)

        

    # for n in range(30, min(length, 100), 10):
        

        # for x in range (0, length-n, 10):
            
        #     print (i,x,n, length,'......')
        #     subdata=data[i, x:x+n, :, :]
        #     padding = np.zeros((300-n, 18, 3))
        #     subdata = np.concatenate((subdata, padding), axis=0)
        #     print (subdata.shape)
        #     subdata= subdata.reshape(1, 300, 18, 3)
        #     subdata=subdata.transpose(0, 3, 1, 2).reshape(subdata.shape[0], 3,300, 18, 1)
        #     subdata= torch.from_numpy(subdata).float()
        #     out, feature= model.extract_feature(subdata.to('cuda'))
        #     index=nList.index(n)
        #     allFeatureList[index].append(feature.cpu().detach().numpy())

# for i in range(len(allFeatureList)):
#     featureList=allFeatureList[i]
#     featureList=np.array(featureList)
#     np.save(featureFolder + 'adl_feature_expand'+str(nList[i])+'_1125_10int.npy', featureList)



data=data.transpose(0, 3, 1, 2).reshape(data.shape[0], 3,300, 18, 1)

data= torch.from_numpy(data).float()
out, feature= model.extract_feature(data.to('cuda'))
# save feature
np.save(featureFolder + 'adl_feature_1125.npy', feature.cpu().detach().numpy())

(40, 300, 18, 3)
0 ......
(143, 18, 3)
(300, 18, 3)
(300, 18, 3)
(300, 18, 3)
(300, 18, 3)
(300, 18, 3)
(300, 18, 3)
(300, 18, 3)
(300, 18, 3)
(300, 18, 3)
(300, 18, 3)
(300, 18, 3)
(300, 18, 3)
(300, 18, 3)
(300, 18, 3)
(300, 18, 3)
1 ......
(149, 18, 3)
(300, 18, 3)
(300, 18, 3)
(300, 18, 3)
(300, 18, 3)
(300, 18, 3)
(300, 18, 3)
(300, 18, 3)
(300, 18, 3)
(300, 18, 3)
(300, 18, 3)
(300, 18, 3)
(300, 18, 3)
(300, 18, 3)
(300, 18, 3)
(300, 18, 3)
2 ......
(180, 18, 3)
(300, 18, 3)
(300, 18, 3)
(300, 18, 3)
(300, 18, 3)
(300, 18, 3)
(300, 18, 3)
(300, 18, 3)
(300, 18, 3)
(300, 18, 3)
(300, 18, 3)
(300, 18, 3)
(300, 18, 3)
(300, 18, 3)
(300, 18, 3)
(300, 18, 3)
(300, 18, 3)
(300, 18, 3)
(300, 18, 3)
3 ......
(150, 18, 3)
(300, 18, 3)
(300, 18, 3)
(300, 18, 3)
(300, 18, 3)
(300, 18, 3)
(300, 18, 3)
(300, 18, 3)
(300, 18, 3)
(300, 18, 3)
(300, 18, 3)
(300, 18, 3)
(300, 18, 3)
(300, 18, 3)
(300, 18, 3)
(300, 18, 3)
4 ......
(180, 18, 3)
(300, 18, 3)
(300, 18, 3)
(300, 18, 3)
(300, 18, 3)
(3

In [19]:
featureFolder='./data/urfall/feature/1125_2048/'
# cerate folder if not exist
if not os.path.exists(featureFolder):
    os.makedirs(featureFolder)
adlSkeleton = './data/urfall/jointNumpy/fall_Data1124.npy'


data=np.load(adlSkeleton, allow_pickle=True)

    
print (data.shape)
# (30, 300, 18, 3)
data=data.transpose(0, 3, 1, 2).reshape(data.shape[0], 3,300, 18, 1)


data= torch.from_numpy(data).float()

out, feature= model.extract_feature(data.to('cuda'))

print (feature.shape)

# save feature
np.save(featureFolder + 'fall_feature_1125.npy', feature.cpu().detach().numpy())

(30, 300, 18, 3)
torch.Size([30, 1, 18, 75, 256])
